In [1]:
!pip install PyPDF2

In [2]:
from PyPDF2 import PdfReader

reader = PdfReader("Ashish_Gupta_Resume_py (1).pdf")
number_of_pages = len(reader.pages)
text=""
for page_num in range(number_of_pages):
    page_text=reader.pages[page_num]
    text+=page_text.extract_text()
  

In [3]:
text

'ASHISH GUPTA\n+91-7887077745 ⋄Noida, IN\nashishguptaji21@gmail.com ⋄linkedin.com/in/Ashish Gupta ⋄Ashish-21CS132.github.io\nEDUCATION\nBTech-Computer Science and Engineering JSS Academy of Technical Education Noida. Expected 2025\nRelevant Coursework: Data Structures, Database Management System, OOP, Design and Analysis of Algorithms,\nOperating System, Machine Learning Techniques\nSKILLS\nLanguages C, C++, Python, Javascript, HTML, CSS, SQL\nFrameworks ReactJS, ExpressJS, Redux Toolkit, Flask, Langchain\nDatabases MySQL, PostgreSQL, MongoDB\nTools and Other Git, GitHub, Insomnia, Postman, OpenAI, LLM\nPython Libraries NumPy, Pandas, Scikit-Learn, Matplotlib, Seaborn, Machine Learning Algorithms\nPROJECTS\nConnectMe (Github)\nReactJS, JavaScript, NodeJS, ExpressJS, MongoDB, Redux Toolkit, TailwindCSS\n•A Full Stack project (MERN) which is a social media website.\n•Enabled Post creation, connection request, update profile, like other’s posts, profile viewing.\nQueryBot (Github)\nPython

In [4]:
import re

def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

In [5]:
process_text=preprocess_text(text)

In [6]:
process_text

'ASHISH GUPTA 917887077745 Noida IN ashishguptaji21gmailcom linkedincominAshish Gupta Ashish21CS132githubio EDUCATION BTechComputer Science and Engineering JSS Academy of Technical Education Noida Expected 2025 Relevant Coursework Data Structures Database Management System OOP Design and Analysis of Algorithms Operating System Machine Learning Techniques SKILLS Languages C C Python Javascript HTML CSS SQL Frameworks ReactJS ExpressJS Redux Toolkit Flask Langchain Databases MySQL PostgreSQL MongoDB Tools and Other Git GitHub Insomnia Postman OpenAI LLM Python Libraries NumPy Pandas ScikitLearn Matplotlib Seaborn Machine Learning Algorithms PROJECTS ConnectMe Github ReactJS JavaScript NodeJS ExpressJS MongoDB Redux Toolkit TailwindCSS A Full Stack project MERN which is a social media website Enabled Post creation connection request update profile like others posts profile viewing QueryBot Github Python GenAI LLM Langchain Gemini Streamlit Jupyter MySQL A GenerativeAI LLM Model It is a Qu

In [7]:
import os

In [15]:
os.environ["GOOGLE_API_KEY"]='google_api_key'

In [9]:
# from langchain_google_genai import ChatGoogleGenerativeAIabs
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="models/text-bison-001")

# llm = ChatGoogleGenerativeAI(model="gemini-pro-vision")

In [10]:
def generate_quiz_questions(input_text):
    prompt = """You are a quiz generator. Given the text below,  create a multiple-choice question with four options (A, B, C, D) and identify the correct answer. Each question should be followed by four possible answers, where one of them is marked as correct.

**Text:** {input_text}

**Instructions:**
1. Read the provided text carefully.
2. Formulate a question that is relevant to the text.
3. Provide four answer options for the question.
4. Clearly identify which option is correct by indicating it as the correct answer.

**Example Output:**

**Question:** What is Python known for according to the text?
(A) Complex syntax
(B) Low-level programming
(C) **Readability and simplicity**
(D) None of the above

---

Please generate five question with four options and indicate the correct answer based on the provided text."""

    response = llm.invoke(prompt)
    return response

In [11]:
def create_quiz_from_pdf():
    
    clean_text = process_text
    quiz_questions = generate_quiz_questions(clean_text)
    return quiz_questions


In [12]:
def format_quiz(raw_quiz_text):
    # Split the input text into lines
    lines = raw_quiz_text.strip().split('\n')
    formatted_quiz = []
    current_question = {}
    
    for line in lines:
        # Identify question lines
        if line.startswith("**Question"):
            # Save the previous question if it exists
            if current_question:
                if "question" in current_question and current_question["options"]:
                    formatted_quiz.append(current_question)
                current_question = {}
            # Start a new question
            current_question["question"] = line.strip().replace("**", "")
            current_question["options"] = []
        # Identify options
        elif line.startswith("(A)") or line.startswith("(B)") or line.startswith("(C)") or line.startswith("(D)"):
            option = line.strip()
            if "**" in option:
                # Remove the asterisks from the correct answer and mark it
                option = option.replace("**", "")
                current_question["answer"] = option
            # Add the option to the list of options
            current_question["options"].append(option)
        # Identify correct answer
        elif line.startswith("Correct answer:"):
            correct_answer = line.strip().split(":")[1].strip()
            current_question["answer"] = correct_answer
    
    # Append the last question if it exists
    if current_question and "question" in current_question and current_question["options"]:
        formatted_quiz.append(current_question)
    
    return formatted_quiz

In [13]:
quiz_questions = create_quiz_from_pdf()
quiz_questions

# save_quiz_to_file(formatted_quiz, output_path)

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


'**Question 1:**\n\nWhat is the most important aspect of a good design?\n(A) **Functionality**\n(B) Aesthetics\n(C) Cost\n(D) Sustainability\n\n**Correct Answer:** A\n\n**Question 2:**\n\nWhich of the following is not a type of computer programming language?\n(A) **Assembly**\n(B) C++\n(C) Java\n(D) Python\n\n**Correct Answer:** D\n\n**Question 3:**\n\nWhat is the capital of the United States?\n(A) **Washington, D.C.**\n(B) New York City\n(C) Los Angeles\n(D) Chicago\n\n**Correct Answer:** A\n\n**Question 4:**\n\nWhat is the highest mountain in the world?\n(A) Mount Everest\n(B) K2\n(C) Mount Kilimanjaro\n(D) Mount Denali\n\n**Correct Answer:** A\n\n**Question 5:**\n\nWho is the current president of the United States?\n(A) **Joe Biden**\n(B) Donald Trump\n(C) Barack Obama\n(D) Bill Clinton\n\n**Correct Answer:** A'

In [14]:
formatted_quiz = format_quiz(quiz_questions)
formatted_quiz

[{'question': 'Question 1:',
  'options': ['(A) Functionality',
   '(B) Aesthetics',
   '(C) Cost',
   '(D) Sustainability'],
  'answer': '(A) Functionality'},
 {'question': 'Question 2:',
  'options': ['(A) Assembly', '(B) C++', '(C) Java', '(D) Python'],
  'answer': '(A) Assembly'},
 {'question': 'Question 3:',
  'options': ['(A) Washington, D.C.',
   '(B) New York City',
   '(C) Los Angeles',
   '(D) Chicago'],
  'answer': '(A) Washington, D.C.'},
 {'question': 'Question 4:',
  'options': ['(A) Mount Everest',
   '(B) K2',
   '(C) Mount Kilimanjaro',
   '(D) Mount Denali']},
 {'question': 'Question 5:',
  'options': ['(A) Joe Biden',
   '(B) Donald Trump',
   '(C) Barack Obama',
   '(D) Bill Clinton'],
  'answer': '(A) Joe Biden'}]